##### 7. Fit a neural network to the Default data. Use a single hidden layer with 10 units, and dropout regularization. Have a look at Labs 10.9.1– 10.9.2 for guidance. Compare the classifcation performance of your model with that of linear logistic regression.

In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from ISLP import load_data

#### Loading in data & cleaning it:

In [2]:
df = load_data('Default')
df

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138947
3,No,No,529.250605,35704.493935
4,No,No,785.655883,38463.495879
...,...,...,...,...
9995,No,No,711.555020,52992.378914
9996,No,No,757.962918,19660.721768
9997,No,No,845.411989,58636.156984
9998,No,No,1569.009053,36669.112365


In [3]:
df.replace({'Yes' : 1, 'No' : 0}, inplace=True)
df

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879
...,...,...,...,...
9995,0,0,711.555020,52992.378914
9996,0,0,757.962918,19660.721768
9997,0,0,845.411989,58636.156984
9998,0,0,1569.009053,36669.112365


In [4]:
from sklearn.model_selection import train_test_split
x = df.drop('default', axis=1)
y = pd.get_dummies(df['default'])
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

### Neural Network:

In [5]:
class Network(nn.Module):
    def __init__(self, input_shape):
        super().__init__()
        self.flatten = nn.Flatten()
        self.sequential = nn.Sequential(nn.Linear(input_shape, 10),
                                        nn.ReLU(),
                                        nn.Dropout(0.4),
                                        nn.Linear(10, 2)
                                        )

    def forward(self, x):
        x = self.flatten(x)
        return self.sequential(x)

In [6]:
from torch.utils.data import TensorDataset
x_train_t = torch.tensor(x_train.values.astype(np.float32))
x_test_t = torch.tensor(x_test.values.astype(np.float32))
y_train_t = torch.tensor(y_train.values.astype(np.float32))
y_test_t = torch.tensor(y_test.values.astype(np.float32))

df_train = TensorDataset(x_train_t, y_train_t)
df_test = TensorDataset(x_test_t, y_test_t)

In [7]:
model = Network(x_train_t.shape[1])

In [8]:
from torchinfo import summary
summary(model, x_train.shape, col_names=['input_size', 'output_size', 'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Network                                  [7500, 3]                 [7500, 2]                 --
├─Flatten: 1-1                           [7500, 3]                 [7500, 3]                 --
├─Sequential: 1-2                        [7500, 3]                 [7500, 2]                 --
│    └─Linear: 2-1                       [7500, 3]                 [7500, 10]                40
│    └─ReLU: 2-2                         [7500, 10]                [7500, 10]                --
│    └─Dropout: 2-3                      [7500, 10]                [7500, 10]                --
│    └─Linear: 2-4                       [7500, 10]                [7500, 2]                 22
Total params: 62
Trainable params: 62
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.47
Input size (MB): 0.09
Forward/backward pass size (MB): 0.72
Params size (MB): 0.00
Estimated Total Size (MB): 0.81

In [9]:
from ISLP.torch import SimpleDataModule, SimpleModule, ErrorTracker, rec_num_workers
from pytorch_lightning.loggers  import CSVLogger
data_module = SimpleDataModule(df_train, df_test, batch_size=50, num_workers=rec_num_workers())
module = SimpleModule.binary_classification(model)
logger = CSVLogger('logs', 'Default')

In [10]:
from pytorch_lightning import Trainer
trainer = Trainer(deterministic=True, max_epochs=30, logger=logger, callbacks=[ErrorTracker()])
trainer.fit(module, datamodule=data_module)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type              | Params
--------------------------------------------
0 | model | Network           | 62    
1 | loss  | BCEWithLogitsLoss | 0     
--------------------------------------------
62        Trainable params
0         Non-trainable params
62        Total params
0.000     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pytorch_lightning/utilities/data.py:104: Total length of `DataLoader` across ranks is zero. Please make sure this was your intention.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


In [11]:
trainer.test(module, datamodule=data_module)

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          0.967199981212616
        test_loss           0.1443472057580948
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.1443472057580948, 'test_accuracy': 0.967199981212616}]

### Logistic Regression:

In [5]:
x = df.drop('default', axis=1)
y = df['default']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [7]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression().fit(x_train, y_train)
logit.score(x_test, y_test)

0.9612